In [ ]:
from PIL import Image, ImageDraw, ImageFont, ExifTags
from datetime import datetime
import os

In [ ]:
def get_exif_datetime(image_path):
    # Open the image
    with Image.open(image_path) as img:
        # Get EXIF data
        exif = img._getexif()

        if not exif:
            return None

        # Iterate through EXIF data and find the date and time
        for tag, value in exif.items():
            decoded_tag = ExifTags.TAGS.get(tag, tag)
            if decoded_tag == 'DateTimeOriginal':
                return value

    return None

In [ ]:
def add_timestamp(image_path, output_path):
    # Get the date and time from EXIF data
    exif_datetime = get_exif_datetime(image_path)

    if not exif_datetime:
        print("EXIF date/time not found in image. Using current date/time instead.")
        from datetime import datetime
        exif_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Load the image
    image = Image.open(image_path)

    # Prepare to draw on the image
    draw = ImageDraw.Draw(image)

    # Specify a font size
    try:
        # Update with the path to your .ttf file and desired size
        font = ImageFont.truetype('/System/Library/Fonts/Supplemental/Courier New.ttf', size=100)
    except IOError:
        # Fallback to default font if .ttf file is not available
        font = ImageFont.load_default()

    # Define the color (yellow in this case)
    color = 'yellow'

    # Position for the timestamp (bottom right corner)
    text_position = (image.width - 1800, image.height - 200)

    # Add the EXIF date/time to the image
    draw.text(text_position, exif_datetime, fill=color, font=font)

    # Save the new image
    image.save(output_path)

In [ ]:
def add_timestamp_to_directory(input_directory):
    for filename in os.listdir(input_directory):
        # Check if the file is an image (e.g., ends with .jpg or .jpeg)
        if filename.lower().endswith(('.jpg', '.jpeg')):
            input_path = os.path.join(input_directory, filename)
            # Create a new filename with "_timestamp" appended
            name, ext = os.path.splitext(filename)
            output_filename = f"{name}_timestamp{ext}"
            output_path = os.path.join(input_directory, output_filename)

            # Add timestamp to the image
            add_timestamp(input_path, output_path)
            print(f"Processed {filename}")

In [ ]:
# Use the current working directory
current_directory = os.getcwd()

# Process all images in the current directory
add_timestamp_to_directory(current_directory)